In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from math import sqrt
from scipy.stats import norm
from scipy.stats import uniform
from sklearn.preprocessing import MinMaxScaler
import scipy
import time
import multiprocessing

import os
#change this to your own path
os.chdir('/Users/robinvogtland/Documents/RV_ETH_CSE_Bachelor/3_Jahr/FS/Bachelor_Thesis/rv_bachelor_thesis/Rough_Bergomi_Experiments/rbergomi')

import numpy as np
from matplotlib import pyplot as plt
from rbergomi import rBergomi 
from utils import bsinv
vec_bsinv = np.vectorize(bsinv)

In [29]:
num_forward_var = 1
num_model_parameters = 3 + num_forward_var
num_strikes = 8
num_maturities = 8


num_input_parameters = num_model_parameters
num_output_parameters = num_maturities*num_strikes
learning_rate = 0.0001
num_steps = 5
batch_size = 10
num_neurons = 40

#initial values
S0 = 1.0
r = 0.00


contract_bounds = np.array([[0.8*S0,1.2*S0],[1,10]]) #bounds for K,T
model_bounds = np.array([[0.1,0.5],[0.5,3],[-0.9,-0.1],[0.01,0.15]]) #bounds for H,eta,rho,lambdas

"""
Note: The grid of stirkes and maturities is equidistant here put could be choosen differently for real world application.
Note: For the code below to striktly follow the bounds specified above make sure that *_distance x num_* is less than half the distance from the highest to lowest * (* = strikes/maturities). 
"""
maturities_distance = (contract_bounds[1,1]-contract_bounds[1,0])/(2*num_maturities) 
strikes_distance = (contract_bounds[0,1]-contract_bounds[0,0])/(2*num_strikes)

strikes = np.linspace(contract_bounds[0,0],contract_bounds[0,0]+num_strikes*strikes_distance,num_strikes)
maturities = np.linspace(contract_bounds[1,0],contract_bounds[1,0]+num_maturities*maturities_distance,num_maturities)

In [30]:
np.random.seed(42)

In [45]:
def reverse_transform_X(X_scaled):
    X = np.zeros(X_scaled.shape)
    for i in range(num_model_parameters):
        X[:,i] = X_scaled[:,i]*(model_bounds[i][1]-model_bounds[i][0]) + model_bounds[i][0]
    for i in range(num_forward_var):
        X[:,i+3] = X_scaled[:,i+3]*(model_bounds[3][1]-model_bounds[3][0]) + model_bounds[3][0]
    return X


def next_batch_rBergomi(batch_size,contract_bounds,model_bounds):
    X_scaled = np.zeros((batch_size,num_input_parameters))
    y = np.zeros((batch_size,num_output_parameters))

    X_scaled[:,0] = uniform.rvs(size=batch_size) #H
    X_scaled[:,1] = uniform.rvs(size=batch_size) #eta
    X_scaled[:,2] = uniform.rvs(size=batch_size) #rho
    lambdas = uniform.rvs(size=(batch_size,num_forward_var))
    for i in range(num_forward_var):
        X_scaled[:,i+3] = lambdas[:,i]

    X = reverse_transform_X(X_scaled)

    for i in range(batch_size):
        for j in range(num_maturities):
            rB = rBergomi.rBergomi(n = 100, N = 30000, T = maturities[j], a = X[i,0]-0.5)

            dW1 = rB.dW1()
            dW2 = rB.dW2()

            Y = rB.Y(dW1)

            dB = rB.dB(dW1, dW2, rho = X[i,2])

            V = rB.V(Y, xi = X[i,3], eta = X[i,1])

            S = rB.S(V, dB) 

            ST = S[:,-1][:,np.newaxis]
            call_payoffs = np.maximum(ST - strikes,0)
            
            call_prices = np.mean(call_payoffs, axis = 0)[:,np.newaxis]
            K = strikes[np.newaxis,:]
            implied_vols = vec_bsinv(call_prices, S0, np.transpose(K), maturities[j])
            
            y[i,j*num_maturities:j*num_maturities+num_strikes] = implied_vols[:,0]
    
    return X_scaled,y

In [47]:
next_batch_rBergomi(1,contract_bounds,model_bounds)

(array([[0.4305529 , 0.00928633, 0.7288886 , 0.40693851]]),
 array([[0.26487127, 0.2629347 , 0.26110934, 0.25948396, 0.25802195,
         0.25680358, 0.25574312, 0.25469034, 0.26991092, 0.26708768,
         0.26462763, 0.26245175, 0.26046486, 0.25866673, 0.25702863,
         0.25550665, 0.25877998, 0.25743541, 0.25616266, 0.25495321,
         0.25380417, 0.25266579, 0.25160516, 0.25061003, 0.26068976,
         0.25897342, 0.25741383, 0.25596733, 0.25468088, 0.25346721,
         0.25230529, 0.25117332, 0.25325768, 0.25204843, 0.25084567,
         0.24976327, 0.24876223, 0.24776613, 0.24682467, 0.24597366,
         0.25039922, 0.24922903, 0.24807051, 0.24697773, 0.24589111,
         0.24485945, 0.24388415, 0.24293259, 0.25179631, 0.25066305,
         0.24958135, 0.24853441, 0.24755091, 0.24662665, 0.24577843,
         0.24500428, 0.26162057, 0.25991405, 0.25830877, 0.2568002 ,
         0.25537058, 0.25409673, 0.25292246, 0.25181221]]))